# Plotting charts

An example how to plot a candlestick chart, also known as OHLCV (open-high-low-close-volume) chart.
Note that volume element may or may not be present.

We will

* Download 1d candle data for all pairs (several dozens of megabytes)

* Extract data for ETH-USDC pair on Uniswap v2

* Plot an interactive OHLC chart using [Plotly](https://pypi.org/project/plotly/)

First let's import libraries and initialise our dataset client.

In [6]:
try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

# Pandas and Plotly are installed as capitalgram dependencies
import pandas as pd
import plotly.graph_objects as go
from capitalgram.client import Capitalgram
from capitalgram.candle import CandleBucket

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Download all 1 day (24h) candles. This dataset is several dozens of megabytes.
You should see a progress bar during the download.

**Note**: We could also query live candle datasets for trading pairs we are interested in.
However, the bundled dataset is faster for the future analysis, as after the download
it will have all candles cached in your local notebook environment.

In [7]:
# Download all 24h candles as Parquet columnar data
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24)

Filter pairs we are interested in.

In [8]:
# Convert PyArrow table to Pandas format to continue working on it
all_candles_dataframe = all_candles.to_pandas()

# To shorten this notebook, we know by hearth that USDC-ETH Uniswap v2 is the pair id numero uno,
# because that what Hayden used to test the production code when he deployed Uniswap v2
pair_id = 1

eth_usdc_pair: pd.DataFrame = all_candles_dataframe.loc[all_candles_dataframe['pair_id'] == pair_id]

print(f"Uniswap v2 ETH-USDC has {len(eth_usdc_pair)} daily candles")


Uniswap v2 ETH-USDC has 434 daily candles


First we have a candlestick chart

In [9]:
fig = go.Figure(data=[go.Candlestick(x=eth_usdc_pair['timestamp'],
                open=eth_usdc_pair['open'],
                high=eth_usdc_pair['high'],
                low=eth_usdc_pair['low'],
                close=eth_usdc_pair['close'])])

fig.show()

Then we have OHCL chart

In [10]:
fig = go.Figure(data=[go.Ohlc(x=eth_usdc_pair['timestamp'],
                open=eth_usdc_pair['open'],
                high=eth_usdc_pair['high'],
                low=eth_usdc_pair['low'],
                close=eth_usdc_pair['close'])])

fig.show()







